In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import wrangle
import prep
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, explained_variance_score
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

wrangle.py functions loaded successfully


ModuleNotFoundError: No module named 'prep'

In [ ]:
df = prep.prep_zillow_data()
df.isna().sum()

In [ ]:
X_train, y_train, X_validate, y_validate, X_test, y_test = wrangle.train_validate_test(df)

In [ ]:
X_train_scaled, X_validate_scaled, X_test_scaled = wrangle.min_max_scale(X_train, X_validate, X_test)

In [ ]:
X_train_scaled

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression

In [ ]:
f_selector = SelectKBest(f_regression, k=5)

In [ ]:
f_selector = f_selector.fit(X_train_scaled, y_train.taxvaluedollarcnt)

In [ ]:
X_train_reduced = f_selector.transform(X_train_scaled)
print(X_train.shape)
print(X_train_reduced.shape)

In [ ]:
f_support = f_selector.get_support()
f_support

In [ ]:
f_feature = X_train_scaled.iloc[:, f_support].columns.tolist()
f_feature

In [ ]:
X_reduced_scaled = X_train_scaled.iloc[:, f_support]
X_reduced_scaled.head()

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

In [ ]:
lm = LinearRegression()

rfe = RFE(lm, 5)

lm_X_rfe = rfe.fit_transform(X_train,y_train) 

In [ ]:
mask = rfe.support_ 
rfe_features = X_train.columns[mask]
print(f'selected {len(rfe_features)} features:', ', '.join(rfe_features))

In [ ]:
lm.fit(lm_X_rfe, y_train)

In [ ]:
print("Linear Model:", lm)

print("intercept: ", lm.intercept_)

print("features: ", rfe_features)

print("coefficients: ", lm.coef_)

In [ ]:
y_train['yhat_lm'] = lm.predict(lm_X_rfe)

y_train.head()

In [ ]:
RMSE_lm = np.sqrt(mean_squared_error(y_train.taxvaluedollarcnt, y_train.yhat_lm))
RMSE_lm

In [ ]:
r2_lm = lm.score(lm_X_rfe, y_train.taxvaluedollarcnt)

r2_lm

In [ ]:
output = "{} = {:.4} + {:.2} * {} + {:.3} * {}".format(
    y_train.columns[0],
    lm.intercept_[0],
    lm.coef_[0][0],
    rfe_features[0],
    lm.coef_[0][1],
    rfe_features[1],
)
output

In [ ]:
lm_X_rfe = rfe.fit_transform(X_validate,y_validate)
mask = rfe.support_ 
rfe_features = X_validate.columns[mask]
print(f'selected {len(rfe_features)} features:', ', '.join(rfe_features))

In [ ]:
y_validate['yhat_lm'] = lm.predict(lm_X_rfe).round(1)

y_validate.head()

In [ ]:
RMSE_lm = np.sqrt(mean_squared_error(y_validate.taxvaluedollarcnt, y_validate.yhat_lm))
RMSE_lm

In [ ]:
r2_lm = lm.score(lm_X_rfe, y_validate.taxvaluedollarcnt)

r2_lm

In [ ]:
output = "{} = {:.4} + {:.2} * {} + {:.3} * {}".format(
    y_train.columns[0],
    lm.intercept_[0],
    lm.coef_[0][0],
    rfe_features[0],
    lm.coef_[0][1],
    rfe_features[1],
)
output

In [ ]:
y_train['yhat_baseline'] = df['taxvaluedollarcnt'].mean()

# compute the RMSE
RMSE_bl = np.sqrt(mean_squared_error(y_train.taxvaluedollarcnt, y_train.yhat_baseline))
print(RMSE_bl)

# no need to compute R-2 because it will be a 0! But we will demonstrate here:
evs = explained_variance_score(y_train.taxvaluedollarcnt, y_train.yhat_baseline)
print(evs)
plt.scatter(y_train.taxvaluedollarcnt, y_train.yhat_lm, label='Linear Regression Model Predictions', marker='o')
plt.scatter(y_train.taxvaluedollarcnt, y_train.yhat_baseline, label='Baseline Predicitions', marker='o')

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly_X_rfe = rfe.fit_transform(X_train,y_train)

poly = PolynomialFeatures(degree=4)
X_poly = poly.fit_transform(poly_X_rfe)

poly.get_feature_names()

In [ ]:
lm_poly = LinearRegression()
lm_poly.fit(X_poly, y_train.taxvaluedollarcnt)
y_train['yhat_poly_1'] = lm_poly.predict(X_poly)

In [ ]:
RMSE_poly = np.sqrt(mean_squared_error(y_train.taxvaluedollarcnt, y_train.yhat_poly_1))
RMSE_poly

In [ ]:
y_train.head()

In [ ]:
plt.figure(figsize=(9, 9))

plt.scatter(y_train.taxvaluedollarcnt, y_train.yhat_lm, label='Linear Regression Model Predictions', marker='o')
plt.scatter(y_train.taxvaluedollarcnt, y_train.yhat_poly_1, label='Polynomial Regression Model Predictions', marker='o')
plt.scatter(y_train.taxvaluedollarcnt, y_train.yhat_baseline, label='Baseline Predicitions', marker='o')

In [ ]:
poly = PolynomialFeatures(degree=3)
X_poly = poly.fit_transform(poly_X_rfe)

In [ ]:
lm_poly = LinearRegression()
lm_poly.fit(X_poly, y_train.taxvaluedollarcnt)
y_train['yhat_poly_2'] = lm_poly.predict(X_poly)

In [ ]:
RMSE_poly = np.sqrt(mean_squared_error(y_train.taxvaluedollarcnt, y_train.yhat_poly_2))
RMSE_poly

In [ ]:
y_train.head()

In [ ]:
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(poly_X_rfe)
lm_poly = LinearRegression()
lm_poly.fit(X_poly, y_train.taxvaluedollarcnt)
y_train['yhat_poly_3'] = lm_poly.predict(X_poly)
RMSE_poly = np.sqrt(mean_squared_error(y_train.taxvaluedollarcnt, y_train.yhat_poly_3))
RMSE_poly

In [ ]:
y_train.head()

In [ ]:
val_poly_X_rfe = rfe.fit_transform(X_validate,y_validate) 

In [ ]:
poly = PolynomialFeatures(degree=4)
X_poly = poly.fit_transform(val_poly_X_rfe)

poly.get_feature_names()

In [ ]:
lm_poly = LinearRegression()
lm_poly.fit(X_poly, y_validate.taxvaluedollarcnt)
y_validate['yhat_poly'] = lm_poly.predict(X_poly).round(1)

In [ ]:
RMSE_poly = np.sqrt(mean_squared_error(y_validate.taxvaluedollarcnt, y_validate.yhat_poly))
RMSE_poly

In [ ]:
y_validate.head()

In [ ]:
plt.scatter(y_validate.taxvaluedollarcnt, y_validate.yhat_poly, label='Polynomial Regression Model Predictions', marker='o')
plt.scatter(y_train.taxvaluedollarcnt, y_train.yhat_baseline, label='Baseline Predicitions', marker='o')

In [ ]:
test_poly_X_rfe = rfe.fit_transform(X_test,y_test) 

In [ ]:
poly = PolynomialFeatures(degree=4)
X_poly = poly.fit_transform(test_poly_X_rfe)

poly.get_feature_names()

In [ ]:
lm_poly = LinearRegression()
lm_poly.fit(X_poly, y_test.taxvaluedollarcnt)
y_test['yhat_poly'] = lm_poly.predict(X_poly).round(1)

In [ ]:
RMSE_poly = np.sqrt(mean_squared_error(y_validate.taxvaluedollarcnt, y_validate.yhat_poly))
RMSE_poly

In [ ]:
y_test.head()

In [ ]:
plt.scatter(y_test.taxvaluedollarcnt, y_test.yhat_poly, label='Polynomial Regression Model Predictions', marker='o')
plt.scatter(y_train.taxvaluedollarcnt, y_train.yhat_baseline, label='Baseline Predicitions', marker='o')